In [1]:
import json
import re
import googlemaps

In [2]:
# Use regular expression to find the text between "Phường " and the next comma
def getWardFromAddress(address_string):
    match = re.search(r'Phường ([^,]*)(?=,)', address_string)
    match2 = re.search(r'Ward ([^,]*)(?=,)', address_string)

    if match:
        ward_info = match.group(1)
    elif match2:
        ward_info = match2.group(1)
    else:
        print("[Regex] Fail to find ward in ", address_string)
        ward_info = ''
    return ward_info

# Use regular expression to find the text between "Quận " and the next comma
def getDistrictFromAddress(address_string):
    match = re.search(r'Quận ([^,]*)(?=,)', address_string)
    match2 = re.search(r'District ([^,]*)(?=,)', address_string)
    match3 = re.search(r'Phường [^,]*, ([^,]*)', address_string)

    if match:
        district = match.group(1)
    elif match2:
        district = match2.group(1)
    elif match3:
        district = match3.group(1)
    else:
        district = ''
        print("[Regex] Fail to find district in ", address_string)
    return district

In [3]:
def get_address_components(latitude, longitude):
    # Replace 'YOUR_API_KEY' with your actual Google Maps API key
    api_key = 'HIDDEN_API_KEY'
    
    gmaps = googlemaps.Client(key=api_key)

    try:
        # Make a reverse geocoding request
        reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

        # get formatted address
        formatted_address = reverse_geocode_result[0]['formatted_address']

        # Extract ward and district from address components
        for result in reverse_geocode_result:
            for addr_component in result['address_components']:
                # Get the ward
                if 'administrative_area_level_3' in addr_component['types']:
                    ward = addr_component['long_name']
                else:
                    print("[Google Maps API] Fail to find ward in ", formatted_address)
                    ward = getWardFromAddress(formatted_address)
                    
                # Get the district
                if 'administrative_area_level_2' in addr_component['types']:
                    district = addr_component['long_name']
                else:
                    print("[Google Maps API] Fail to find district in ", formatted_address)
                    district = getDistrictFromAddress(formatted_address)

            # Break the loop if both ward and district are found
            if 'ward' in locals() and 'district' in locals():
                break

        return  formatted_address, ward, district

    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [4]:
def main():
    # Replace 'your-file.csv' with the path to your CSV file
    file_path = 'new points.csv'
    output_data = []

    # Read the CSV file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            columns = line.split(',')
            title = columns[0]
            latitude = float(columns[1])
            longitude = float(columns[2])
            formatted_address, ward, district = get_address_components(latitude, longitude)
            if (ward == None or district == None):
                print("Error no ward district at: ", title)
                continue
            output = {
                'title': title,
                'coords': {
                    'lat': latitude,
                    'lng': longitude
                },
                'address': formatted_address,
                'ward': ward,
                'district': district
            }
            output_data.append(output)

    # Write the output data to a JSON file with UTF-8 encoding
    with open('full2.json', 'w', encoding='utf-8') as json_file:
        json.dump(output_data, json_file, ensure_ascii=False)

In [5]:
main()

[Google Maps API] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Regex] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Google Maps API] Fail to find district in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Google Maps API] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Regex] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Google Maps API] Fail to find district in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Google Maps API] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Regex] Fail to find ward in  192 Hẻm 134/1 Cách Mạng Tháng 8, Khu phố 4, Quận 3, Thành phố Hồ Chí Minh, Vietnam
[Google Maps API] Fail to find distric